In [1]:
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import os

import training as methods

In [4]:
####################
# LOADING THE DATA #
####################
DATA_DIR = "../segmented_data/"
SUBJECTS = ['AT', 'EL', 'HH', 'MS', 'RB', 'RL', 'TT']
SCENES = ['FlatWalkStraight']
TRIALS = ('Bw stop FW walking_3')

dfs = methods.read_gait_cycles(SUBJECTS, SCENES, TRIALS)

Reading Bw stop FW walking_3.csv
Reading Bw stop FW walking_3_badForcePlate.csv
Reading Bw stop FW walking_3.csv
Reading Bw stop FW walking_3_badForcePlate.csv
Reading Bw stop FW walking_3.csv
Reading Bw stop FW walking_3.csv
Reading Bw stop FW walking_3.csv


In [ ]:
def draw_line_chart(label, df):
    fig = plt.figure(figsize=(20, 3))
    gs = fig.add_gridspec(1, 2, hspace=0.22)
    axs = gs.subplots(sharex=True)
    fig.suptitle(label)

    x = df.index
    y = df['Fz_l']
    y3 = df['Ftot_l']
    axs[0].plot(x, y)
    axs[0].plot(x, y3)

    y4 = df['Fz_r']
    y5 = df['Ftot_r']
    axs[1].plot(x, y4)
    axs[1].plot(x, y5)
    
    cond1 = df['valid_mask_feet']  == 1
    cond2 = df['correct_mask_ins'] == 1
    
    cond3_l = (df['fp1_l'] != 1) # | (df['Fz_l'] == 0) 
    cond3_r = (df['fp1_r'] != 1) # | (df['Fz_l'] == 0) 
    
    cond4_l = (df['Ftot_l'] < 0.05) | (df['Fz_l'] != 0)
    cond4_r = (df['Ftot_r'] < 0.05) | (df['Fz_r'] != 0)
    
    
    for ax in axs:
        ax.fill_between(x, 0, 1, where=~cond1, color='green', alpha=0.1, transform=ax.get_xaxis_transform())
        ax.fill_between(x, 0, 1, where=~cond2, color='orange', alpha=0.1, transform=ax.get_xaxis_transform())

    
    axs[0].fill_between(x, 0, 1, where=~cond3_l, color='red', alpha=0.1, transform=axs[0].get_xaxis_transform())
    axs[1].fill_between(x, 0, 1, where=~cond3_r, color='red', alpha=0.1, transform=axs[1].get_xaxis_transform())

    axs[0].fill_between(x, 0, 1, where=~cond4_l, color='blue', alpha=0.1, transform=axs[0].get_xaxis_transform())
    axs[1].fill_between(x, 0, 1, where=~cond4_r, color='blue', alpha=0.1, transform=axs[1].get_xaxis_transform())
    
    # Trimming gait cycles
    first_l = df[(df['Fz_l'] != 0)].index.min()
    first_r = df[(df['Fz_r'] != 0)].index.min()

    if(first_l != df.index.min()):
        test_l = df[(df.index < first_l - 5)].index[-1]
        axs[0].axvspan(df.index.min(), test_l, alpha=0.1, color='grey')

    if(first_r != df.index.min()):
        test_r = df[(df.index < first_r - 5)].index[-1]
        axs[1].axvspan(df.index.min(), test_r, alpha=0.1, color='grey')
    
    
    last_l = df[(df['Fz_l'] != 0)].index.max()
    last_r = df[(df['Fz_r'] != 0)].index.max()
    
    if(last_l != df.index.max()):
        test_l = df[(df.index > last_l + 5)].index[0]
        axs[0].axvspan(test_l, df.index.max(), alpha=0.1, color='grey')

    if(last_r != df.index.max()):
        test_r = df[(df.index > last_r + 5)].index[0]
        axs[1].axvspan(test_r, df.index.max(), alpha=0.1, color='grey')
    
    fig.savefig('test.svg', format='svg')


for df in dfs:
    label = df['trial'].iloc[0]
    draw_line_chart(label, df)